In [ ]:
# Install required packages (uncomment if needed)
# !pip install torch torchvision albumentations opencv-python matplotlib tqdm

import copy
import os
import random
import shutil
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from urllib.request import urlretrieve

import albumentations as A
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim
from albumentations.pytorch import ToTensorV2
from torch.backends import cudnn
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from tqdm import tqdm

# Performance optimizations
cudnn.benchmark = True

# Set random seeds for reproducibility
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"Albumentations version: {A.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


In [ ]:
# Configuration parameters
params = {
    "model": "resnet50",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "lr": 0.001,
    "batch_size": 64,
    "num_workers": 4,
    "epochs": 10,
    "image_size": 128,
    "val_split": 0.2,
    "test_size": 10,
    "seed": 42,
}

# Data directory
dataset_directory = Path.home() / "datasets" / "cats-vs-dogs"
dataset_directory.mkdir(parents=True, exist_ok=True)

print(f"Configuration:")
print(f"  Model: {params['model']}")
print(f"  Device: {params['device']}")
print(f"  Batch size: {params['batch_size']}")
print(f"  Learning rate: {params['lr']}")
print(f"  Epochs: {params['epochs']}")
print(f"  Dataset directory: {dataset_directory}")


In [ ]:
class TqdmUpTo(tqdm):
    """Progress bar for downloads."""
    def update_to(self, b: int = 1, bsize: int = 1, tsize: Optional[int] = None) -> None:
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url: str, filepath: Path) -> None:
    """Download file with progress bar."""
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    if filepath.exists():
        print(f"File already exists at {filepath}. Skipping download.")
        return

    with TqdmUpTo(unit="B", unit_scale=True, unit_divisor=1024, miniters=1, 
                  desc=filepath.name) as t:
        urlretrieve(url, filename=str(filepath), reporthook=t.update_to)
        t.total = t.n

def extract_archive(filepath: Path) -> Path:
    """Extract zip archive."""
    extract_dir = filepath.parent
    shutil.unpack_archive(str(filepath), str(extract_dir))
    return extract_dir / "PetImages"

# Download and extract dataset
dataset_url = "https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip"
zip_path = dataset_directory / "kagglecatsanddogs_5340.zip"

print("Downloading dataset...")
download_url(dataset_url, zip_path)

print("Extracting dataset...")
data_dir = extract_archive(zip_path)
print(f"Dataset extracted to: {data_dir}")


In [ ]:
def validate_images(data_dir: Path) -> Tuple[List[Path], List[str]]:
    """Validate and return valid image paths with labels."""
    cat_dir = data_dir / "Cat"
    dog_dir = data_dir / "Dog"
    
    valid_images = []
    labels = []
    
    # Process cat images
    print("Validating cat images...")
    for img_path in tqdm(list(cat_dir.glob("*.jpg"))):
        if cv2.imread(str(img_path)) is not None:
            valid_images.append(img_path)
            labels.append("Cat")
    
    # Process dog images  
    print("Validating dog images...")
    for img_path in tqdm(list(dog_dir.glob("*.jpg"))):
        if cv2.imread(str(img_path)) is not None:
            valid_images.append(img_path)
            labels.append("Dog")
    
    print(f"\nFound {len(valid_images)} valid images")
    print(f"  - Cats: {labels.count('Cat')}")
    print(f"  - Dogs: {labels.count('Dog')}")
    
    return valid_images, labels

# Validate images
image_paths, labels = validate_images(data_dir)


In [ ]:
def split_dataset(
    image_paths: List[Path], 
    labels: List[str], 
    val_split: float = 0.2, 
    test_size: int = 10, 
    seed: int = 42
) -> Tuple[List[Path], List[Path], List[Path]]:
    """Split dataset into train, validation, and test sets."""
    random.seed(seed)
    
    # Shuffle data together
    combined = list(zip(image_paths, labels))
    random.shuffle(combined)
    image_paths, labels = zip(*combined)
    
    # Calculate splits
    total_size = len(image_paths)
    test_size = min(test_size, total_size // 10)
    val_size = int((total_size - test_size) * val_split)
    train_size = total_size - val_size - test_size
    
    # Split paths
    train_paths = list(image_paths[:train_size])
    val_paths = list(image_paths[train_size:train_size + val_size])
    test_paths = list(image_paths[train_size + val_size:])
    
    print(f"Dataset split:")
    print(f"  - Train: {len(train_paths)}")
    print(f"  - Validation: {len(val_paths)}")
    print(f"  - Test: {len(test_paths)}")
    
    return train_paths, val_paths, test_paths

# Split the dataset
train_paths, val_paths, test_paths = split_dataset(
    image_paths, labels, 
    val_split=params["val_split"], 
    test_size=params["test_size"], 
    seed=params["seed"]
)


In [ ]:
def show_dataset_samples(image_paths: List[Path], num_samples: int = 10):
    """Display sample images from the dataset."""
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.ravel()
    
    sample_paths = random.sample(image_paths, min(num_samples, len(image_paths)))
    
    for idx, img_path in enumerate(sample_paths):
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(img)
        axes[idx].set_title(f"{img_path.parent.name}\nShape: {img.shape}", fontsize=10)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# Show sample training images
print("Sample training images:")
show_dataset_samples(train_paths)


In [ ]:
# Training augmentations - diverse transformations for better generalization
train_transform = A.Compose([
    A.SmallestMaxSize(max_size=160),
    A.Affine(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomCrop(height=128, width=128),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Validation augmentations - minimal and deterministic
val_transform = A.Compose([
    A.SmallestMaxSize(max_size=160),
    A.CenterCrop(height=128, width=128),  # Deterministic crop
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

print("Augmentation pipelines created!")
print("\nTraining augmentations:")
print("  - SmallestMaxSize: Resizes while maintaining aspect ratio")
print("  - Affine: Geometric transformations (rotation, shift, scale)")
print("  - RandomCrop: Spatial variations")
print("  - ColorJitter: Color channel adjustments")
print("  - RandomBrightnessContrast: Lighting variations")
print("  - Normalize: ImageNet normalization")
print("\nValidation augmentations:")
print("  - SmallestMaxSize: Consistent resizing")
print("  - CenterCrop: Deterministic cropping")
print("  - Normalize: Same normalization as training")


In [ ]:
def show_augmented_samples(image_path: Path, num_samples: int = 6):
    """Show original and augmented versions of an image."""
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    # Show original
    axes[0].imshow(img)
    axes[0].set_title("Original", fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Create transform without normalization for visualization
    vis_transform = A.Compose([
        A.SmallestMaxSize(max_size=160),
        A.Affine(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=128, width=128),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
    ])
    
    # Show augmented versions
    for idx in range(1, num_samples):
        augmented = vis_transform(image=img)["image"]
        axes[idx].imshow(augmented)
        axes[idx].set_title(f"Augmented {idx}", fontsize=12)
        axes[idx].axis('off')
    
    plt.suptitle("Augmentation Examples", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Show augmentation examples
print("Augmentation examples for a sample image:")
show_augmented_samples(train_paths[0])


In [ ]:
class CatsVsDogsDataset(Dataset):
    """PyTorch dataset for cats vs dogs classification."""
    
    def __init__(self, image_paths: List[Path], transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self) -> int:
        return len(self.image_paths)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, float]:
        img_path = self.image_paths[idx]
        
        # Load image
        image = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Get label from parent directory name
        label = 1.0 if img_path.parent.name == "Cat" else 0.0
        
        # Apply transformations
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label

# Create datasets
train_dataset = CatsVsDogsDataset(train_paths, transform=train_transform)
val_dataset = CatsVsDogsDataset(val_paths, transform=val_transform)
test_dataset = CatsVsDogsDataset(test_paths, transform=val_transform)  # Use val transform for test

print(f"Dataset sizes:")
print(f"  Training: {len(train_dataset)} images")
print(f"  Validation: {len(val_dataset)} images")
print(f"  Test: {len(test_dataset)} images")


In [ ]:
# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=params["batch_size"],
    shuffle=True,
    num_workers=params["num_workers"],
    pin_memory=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=params["batch_size"],
    shuffle=False,
    num_workers=params["num_workers"],
    pin_memory=True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=params["batch_size"],
    shuffle=False,
    num_workers=params["num_workers"],
    pin_memory=True,
)

print(f"DataLoader configuration:")
print(f"  Batch size: {params['batch_size']}")
print(f"  Number of workers: {params['num_workers']}")
print(f"  Training batches: {len(train_loader)}")
print(f"  Validation batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")


In [ ]:
# Create model
model = getattr(models, params["model"])(pretrained=False, num_classes=1)
model = model.to(params["device"])

# Loss function for binary classification
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

# Learning rate scheduler (optional but recommended)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

print(f"Model: {params['model']}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"Loss function: BCEWithLogitsLoss")
print(f"Optimizer: Adam (lr={params['lr']})")


In [ ]:
class MetricMonitor:
    """Helper class to track and display training metrics."""
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]
        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [f"{metric_name}: {metric['avg']:.{self.float_precision}f}"
             for (metric_name, metric) in self.metrics.items()]
        )


In [ ]:
def calculate_accuracy(output, target):
    """Calculate accuracy for binary classification."""
    prediction = torch.sigmoid(output) >= 0.5
    correct = (prediction == target).sum().item()
    accuracy = correct / output.shape[0]
    return accuracy

print("Helper functions created successfully!")


In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, params):
    """Train the model for one epoch."""
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader, desc=f"Epoch {epoch} [Train]")
    
    for _, (images, target) in enumerate(stream, start=1):
        images = images.to(params["device"], non_blocking=True)
        target = target.to(params["device"], non_blocking=True).float().view(-1, 1)
        
        # Forward pass
        output = model(images)
        loss = criterion(output, target)
        accuracy = calculate_accuracy(output, target)
        
        # Update metrics
        metric_monitor.update("Loss", loss.item())
        metric_monitor.update("Accuracy", accuracy)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update progress bar
        stream.set_postfix_str(str(metric_monitor))
    
    return metric_monitor.metrics["Loss"]["avg"], metric_monitor.metrics["Accuracy"]["avg"]


In [ ]:
def validate(val_loader, model, criterion, epoch, params):
    """Validate the model for one epoch."""
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader, desc=f"Epoch {epoch} [Val]")
    
    with torch.inference_mode():
        for _, (images, target) in enumerate(stream, start=1):
            images = images.to(params["device"], non_blocking=True)
            target = target.to(params["device"], non_blocking=True).float().view(-1, 1)
            
            # Forward pass
            output = model(images)
            loss = criterion(output, target)
            accuracy = calculate_accuracy(output, target)
            
            # Update metrics
            metric_monitor.update("Loss", loss.item())
            metric_monitor.update("Accuracy", accuracy)
            
            # Update progress bar
            stream.set_postfix_str(str(metric_monitor))
    
    return metric_monitor.metrics["Loss"]["avg"], metric_monitor.metrics["Accuracy"]["avg"]


In [ ]:
# Training history storage
history = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}

# Best model tracking
best_val_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

# Training loop
print("Starting training...")
print("-" * 50)

for epoch in range(1, params["epochs"] + 1):
    # Train
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, params)
    
    # Validate
    val_loss, val_acc = validate(val_loader, model, criterion, epoch, params)
    
    # Update history
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)
    
    # Update learning rate
    scheduler.step(val_loss)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"  → New best model! Val Acc: {val_acc:.4f}")
    
    # Print epoch summary
    print(f"\nEpoch {epoch}/{params['epochs']}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    print("-" * 50)

# Load best model weights
model.load_state_dict(best_model_wts)
print(f"\nTraining completed! Best validation accuracy: {best_val_acc:.4f}")


In [ ]:
def plot_training_history(history: dict):
    """Plot training and validation metrics."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot loss
    epochs = range(1, len(history["train_loss"]) + 1)
    ax1.plot(epochs, history["train_loss"], 'bo-', label="Train Loss")
    ax1.plot(epochs, history["val_loss"], 'ro-', label="Val Loss")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.set_title("Training and Validation Loss")
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot accuracy
    ax2.plot(epochs, history["train_acc"], 'bo-', label="Train Accuracy")
    ax2.plot(epochs, history["val_acc"], 'ro-', label="Val Accuracy")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy")
    ax2.set_title("Training and Validation Accuracy")
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot the training history
plot_training_history(history)

# Print final metrics
print(f"\nFinal Training Metrics:")
print(f"  Final Train Loss: {history['train_loss'][-1]:.4f}")
print(f"  Final Train Accuracy: {history['train_acc'][-1]:.4f}")
print(f"  Final Val Loss: {history['val_loss'][-1]:.4f}")
print(f"  Final Val Accuracy: {history['val_acc'][-1]:.4f}")
print(f"  Best Val Accuracy: {best_val_acc:.4f}")


In [ ]:
def evaluate_model(model, test_loader, device):
    """Evaluate model on test set."""
    model.eval()
    predictions = []
    actuals = []
    
    with torch.inference_mode():
        for images, targets in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True).float().view(-1, 1)
            
            outputs = model(images)
            preds = (torch.sigmoid(outputs) >= 0.5).float()
            
            predictions.extend(preds.cpu().numpy())
            actuals.extend(targets.cpu().numpy())
    
    predictions = np.array(predictions).flatten()
    actuals = np.array(actuals).flatten()
    
    # Calculate metrics
    accuracy = (predictions == actuals).mean()
    
    return predictions, actuals, accuracy

# Evaluate on test set
test_predictions, test_actuals, test_accuracy = evaluate_model(model, test_loader, params["device"])

print(f"Test Set Performance:")
print(f"  Test Accuracy: {test_accuracy:.4f}")
print(f"  Correct Predictions: {int((test_predictions == test_actuals).sum())}/{len(test_actuals)}")


In [ ]:
def display_predictions(image_paths, predictions, cols=5):
    """Display images with predicted labels."""
    rows = len(image_paths) // cols
    if len(image_paths) % cols != 0:
        rows += 1
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 3))
    
    # Flatten axes array for easier indexing
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, image_path in enumerate(image_paths):
        row = i // cols
        col = i % cols
        
        # Load and display image
        image = cv2.imread(str(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Get true and predicted labels
        true_label = image_path.parent.name
        predicted_label = "Cat" if predictions[i] == 1.0 else "Dog"
        
        # Set color based on correctness
        color = "green" if true_label == predicted_label else "red"
        
        axes[row, col].imshow(image)
        axes[row, col].set_title(f"True: {true_label}\nPred: {predicted_label}", 
                                 color=color, fontsize=10)
        axes[row, col].axis('off')
    
    # Hide any remaining empty subplots
    for i in range(len(image_paths), rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].axis('off')
    
    plt.suptitle("Test Set Predictions (Green=Correct, Red=Wrong)", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Display test predictions
print("Test Set Predictions:")
display_predictions(test_paths, test_predictions)


In [ ]:
def predict_single_image(model, image_path, transform, device):
    """Make prediction on a single image."""
    # Load and preprocess image
    image = cv2.imread(str(image_path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Apply transformations
    transformed = transform(image=image_rgb)["image"]
    
    # Add batch dimension and move to device
    input_tensor = transformed.unsqueeze(0).to(device)
    
    # Make prediction
    model.eval()
    with torch.inference_mode():
        output = model(input_tensor)
        probability = torch.sigmoid(output).item()
        prediction = "Cat" if probability > 0.5 else "Dog"
        confidence = probability if probability > 0.5 else 1 - probability
    
    return prediction, confidence, image_rgb

# Example: Predict on a random test image
sample_image_path = random.choice(test_paths)
prediction, confidence, image = predict_single_image(model, sample_image_path, val_transform, params["device"])

# Display result
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.title(f"Prediction: {prediction} (Confidence: {confidence:.2%})\nTrue Label: {sample_image_path.parent.name}", 
          fontsize=12)
plt.axis('off')
plt.show()

print(f"Single Image Prediction:")
print(f"  Image: {sample_image_path.name}")
print(f"  True Label: {sample_image_path.parent.name}")
print(f"  Predicted: {prediction}")
print(f"  Confidence: {confidence:.2%}")


In [ ]:
# Save the model
save_path = Path("cats_dogs_classifier.pth")

# Save model state dict with metadata
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'best_val_acc': best_val_acc,
    'params': params,
    'history': history,
}, save_path)

print(f"Model saved to: {save_path}")
print(f"File size: {save_path.stat().st_size / 1024 / 1024:.2f} MB")

# Example: Load the model
def load_model(model_path, device):
    """Load a saved model."""
    checkpoint = torch.load(model_path, map_location=device)
    
    # Create model
    loaded_model = getattr(models, checkpoint['params']['model'])(pretrained=False, num_classes=1)
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    loaded_model = loaded_model.to(device)
    loaded_model.eval()
    
    print(f"Model loaded successfully!")
    print(f"Best validation accuracy: {checkpoint['best_val_acc']:.4f}")
    
    return loaded_model, checkpoint['history'], checkpoint['params']

# Test loading
loaded_model, loaded_history, loaded_params = load_model(save_path, params["device"])


In [ ]:
class MetricMonitor:
    """Helper class to track and display training metrics."""
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]
        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                f"{metric_name}: {metric['avg']:.{self.float_precision}f}"
                for (metric_name, metric) in self.metrics.items()
            ]
        )

def calculate_accuracy(output, target):
    """Calculate accuracy for binary classification."""
    prediction = torch.sigmoid(output) >= 0.5
    correct = (prediction == target).sum().item()
    accuracy = correct / output.shape[0]
    return accuracy

class EarlyStopping:
    """Early stopping to prevent overfitting."""
    def __init__(self, patience: int = 5, min_delta: float = 0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    
    def __call__(self, val_loss: float) -> bool:
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
        
        return self.early_stop

print("Helper functions created successfully!")
